## Preprocessing

In [1]:
pip install numpy --upgrade


Note: you may need to restart the kernel to use updated packages.


In [2]:
import tensorflow as tf
import numpy as np

print("TensorFlow version:", tf.__version__)
print("NumPy version:", np.__version__)


C:\Users\Brian Haynes\anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.25.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


TensorFlow version: 2.13.0
NumPy version: 1.25.1


In [3]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
new_df = application_df.drop(columns=['EIN','NAME'])

In [5]:
# Determine the number of unique values in each column.
new_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [6]:
# Look at APPLICATION_TYPE value counts for binning
types = new_df['APPLICATION_TYPE'].value_counts()
types

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = [index for index,row in types.items() if row < 1000]

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64

In [8]:
# Look at CLASSIFICATION value counts for binning
class_types = new_df['CLASSIFICATION'].value_counts()
class_types

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [23]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = [index for index,row in class_types.items() if row < class_types.mean()]

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [24]:
# Convert categorical data to numeric with `pd.get_dummies`
dummies = pd.get_dummies(new_df[['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE',
       'ORGANIZATION', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS']])
numbers_df = pd.concat([new_df[['STATUS', 'ASK_AMT']], dummies], axis=1)
numbers_df

,STATUS,ASK_AMT,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [25]:
# Split our preprocessed data into our features and target arrays
X = numbers_df
y = new_df['IS_SUCCESSFUL']
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    )

In [26]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [27]:
input_features = len( X_train_scaled[0])

num_classes = 2  


# Define the deep neural network model
nn = tf.keras.models.Sequential()

# First hidden layer with 80 nodes and ReLU activation
nn.add(tf.keras.layers.Dense(80, activation='relu', input_shape=(input_features,)))

# Second hidden layer with 30 nodes and ReLU activation
nn.add(tf.keras.layers.Dense(30, activation='relu'))

# Output layer with num_classes nodes and sigmoid activation (for binary classification)
nn.add(tf.keras.layers.Dense(num_classes, activation='sigmoid'))

# Check the structure of the model
nn.summary()

# Compile the model
nn.compile(loss='binary_crossentropy', 
           optimizer='adam',            
           metrics=['accuracy'])        



Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 80)                9360      
                                                                 
 dense_5 (Dense)             (None, 30)                2430      
                                                                 
 dense_6 (Dense)             (None, 2)                 62        
                                                                 
Total params: 11852 (46.30 KB)
Trainable params: 11852 (46.30 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [28]:

# Compile the model
nn.compile(loss='binary_crossentropy', 
           optimizer='adam',            
           metrics=['accuracy'])        


In [29]:
# Modify the output layer to have a single node with 'sigmoid' activation
nn.add(tf.keras.layers.Dense(1, activation='sigmoid'))


In [30]:
import numpy as np

# Assuming your X_train_scaled has a shape of (None, 116) and you want to reshape it to (None, 10)
X_train_reshaped = X_train_scaled[:, :116]  # Extract the first 10 features from each sample

# Verify the new shape
print(X_train_reshaped.shape)  # Should now have a shape of (None, 10)



(25724, 116)


In [31]:

# Assuming you have already defined and compiled your model as 'nn'
# Make sure to add the appropriate layers and compile the model before this step

# Train the model
epochs = 100
batch_size = 32

# Use the fit() function to start the training process
history = nn.fit(X_train_reshaped, y_train, validation_split=0.15, epochs=epochs, batch_size=batch_size)

# The 'history' variable will store information about the training process (loss and metrics) over epochs
# You can use this information to visualize the training progress and make improvements if needed



Epoch 1/100
684/684 [==============================] - 2s 2ms/step - loss: 0.6026 - accuracy: 0.7086 - val_loss: 0.5800 - val_accuracy: 0.7357
Epoch 2/100
684/684 [==============================] - 1s 2ms/step - loss: 0.5757 - accuracy: 0.7298 - val_loss: 0.5684 - val_accuracy: 0.7334
Epoch 3/100
684/684 [==============================] - 1s 2ms/step - loss: 0.5708 - accuracy: 0.7308 - val_loss: 0.5678 - val_accuracy: 0.7308
Epoch 4/100
684/684 [==============================] - 1s 2ms/step - loss: 0.5653 - accuracy: 0.7323 - val_loss: 0.5598 - val_accuracy: 0.7302
Epoch 5/100
684/684 [==============================] - 1s 2ms/step - loss: 0.5587 - accuracy: 0.7323 - val_loss: 0.5557 - val_accuracy: 0.7323
Epoch 6/100
684/684 [==============================] - 1s 2ms/step - loss: 0.5550 - accuracy: 0.7342 - val_loss: 0.5520 - val_accuracy: 0.7352
Epoch 7/100
684/684 [==============================] - 1s 2ms/step - loss: 0.5529 - accuracy: 0.7339 - val_loss: 0.5508 - val_accuracy: 0.7336

Epoch 58/100
684/684 [==============================] - 1s 2ms/step - loss: 0.5352 - accuracy: 0.7398 - val_loss: 0.5500 - val_accuracy: 0.7318
Epoch 59/100
684/684 [==============================] - 1s 2ms/step - loss: 0.5351 - accuracy: 0.7405 - val_loss: 0.5481 - val_accuracy: 0.7321
Epoch 60/100
684/684 [==============================] - 1s 2ms/step - loss: 0.5354 - accuracy: 0.7406 - val_loss: 0.5488 - val_accuracy: 0.7292
Epoch 61/100
684/684 [==============================] - 1s 2ms/step - loss: 0.5350 - accuracy: 0.7408 - val_loss: 0.5502 - val_accuracy: 0.7326
Epoch 62/100
684/684 [==============================] - 1s 2ms/step - loss: 0.5345 - accuracy: 0.7403 - val_loss: 0.5492 - val_accuracy: 0.7334
Epoch 63/100
684/684 [==============================] - 1s 2ms/step - loss: 0.5352 - accuracy: 0.7399 - val_loss: 0.5472 - val_accuracy: 0.7331
Epoch 64/100
684/684 [==============================] - 1s 2ms/step - loss: 0.5346 - accuracy: 0.7404 - val_loss: 0.5500 - val_accuracy:

In [32]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5540 - accuracy: 0.7298 - 268ms/epoch - 1000us/step
Loss: 0.5540100336074829, Accuracy: 0.7297959327697754


In [33]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity_Optimization.h5')

C:\Users\Brian Haynes\anaconda3\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
